___
# Ciência dos Dados - PROJETO 3 - INSPER

___
## Vitor Pereira

## Rafael Almada


___

## INTRODUÇÃO

O objetivo do projeto é criar um modelo preditivo conduzido por uma análise de dados a partir de um dataset escolhido, e a partir de variáveis qualitativas e quantitativas correlacionas, estimar o valor de uma variável qualitativa. Para tanto nós decidimos fazer uma análise preditiva da variável quantitativa renda, baseando-se no uso da PNAD 2015 (dataset utilizado no projeto 1), utilizando as variáveis explicativas que a dupla julgou relevantes: Estado de nascimento, sexo, idade, cor ou raça, grau de alfabetização, ter utilizado internet nos últimos 12 meses, estar empregado ou não e rendimento no mês da pesquisa. 
A Partir dessas variaveis pretendemos analisar a partir de uma regressão linear, a dispersão dessas variáveis explicativas em relação a renda, ou seja, o quanto essas variáveis selecionadas se relacionam com a renda de uma pessoa que respondeu a pesquisa. Feita a regressão linear vamos comparar os valores obtidos de renda ao rendimento mensal domiciliar per capita de pessoas aleatórias para validar a qualidade da predição. 

___
## MINERANDO DADOS e CARACTERÍSTICAS DO DATASET

[Detalhe aqui as características da base de dados além da análise descritiva.]

In [10]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np


% matplotlib inline
#import pandas as pd
#import numpy as np
import math as math
from scipy import stats
#import matplotlib.pyplot as plt
import statsmodels.api as sm

In [11]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

valores=pd.read_csv('PES2015csv.zip', compression='zip', header=0, sep=',', quotechar='"')


dados=pd.DataFrame({

    'Estado' :valores.UF,
    'Sexo' :valores.V0302,
    'Idade' :valores.V8005,
    'Cor_ou_raca':valores.V0404,
    'Sabe_ler_e_escrever' :valores.V0601,
    'renda' :valores.V4720,
    'Rendimento_mensal_domiciliar_per_capita' :valores.V4742
    })



dados=dados[['Estado','Sexo','Idade','Cor_ou_raca','Sabe_ler_e_escrever','renda']]


dados.Sexo=dados.Sexo.astype('category')
dados.Sexo.cat.categories=(['Masculino','Feminino'])

dados.Cor_ou_raca=dados.Cor_ou_raca.astype('category')
dados.Cor_ou_raca.cat.categories=(['Branca','Preta','Amarela','Parda','Indígena','Sem declaração'])

dados.Sabe_ler_e_escrever=dados.Sabe_ler_e_escrever.astype('category')
dados.Sabe_ler_e_escrever.cat.categories=('Sim','Não')


def Estados(linha):
    if linha>=11 and linha<=17:
        return 'Norte'
    if linha>=21 and linha<=29:
        return 'Nordeste'
    if linha>=31 and linha<=35:
        return 'Sudeste'
    if linha>=41 and linha<=43:
        return 'Sul'
    if linha>=50 and linha<=53:
        return 'Centro-Oeste'
    
dados.Estado=dados.Estado.apply(Estados)

#dados=dados.dropna()

#dados=dados[['renda']]

dados
#.rendimento.mean()

Esperamos trabalhar no diretório
C:\Users\vitor\Documents\DP dados


,Estado,Sexo,Idade,Cor_ou_raca,Sabe_ler_e_escrever,renda
0,Norte,Masculino,23,Indígena,Sim,800.0
1,Norte,Feminino,23,Preta,Sim,1150.0
2,Norte,Feminino,35,Indígena,Sim,880.0
3,Norte,Masculino,34,Indígena,Sim,3000.0
4,Norte,Feminino,11,Indígena,Sim,0.0
5,Norte,Feminino,7,Indígena,Sim,NaN
6,Norte,Feminino,4,Indígena,Não,NaN
7,Norte,Feminino,18,Indígena,Sim,880.0
8,Norte,Masculino,46,Preta,Sim,3500.0
9,Norte,Feminino,81,Indígena,Sim,2300.0


### BASE DE DADOS

In [17]:
#divisao teste e treinamento
from sklearn.model_selection import train_test_split

dados_treino, dados_teste = train_test_split(dados,test_size=0.1)



In [21]:

Y = dados_treino['renda']
X = dados_treino[['Estado','Sexo','Idade','Cor_ou_raca','Sabe_ler_e_escrever']]
X = sm.add_constant(X)
model = sm.OLS(Y,X)
results = model.fit()
results.summary()



341687    1700.0
48317        0.0
218514       0.0
320312       0.0
188970    1600.0
281401    1500.0
289961    4000.0
254211       0.0
338437    2500.0
110160       NaN
279594       0.0
269266       0.0
325773       0.0
158701     800.0
151274    1100.0
250265       0.0
33876        0.0
101303     858.0
174976       0.0
151316     750.0
154441     900.0
8498         0.0
122135       NaN
47563      788.0
278600    1500.0
322184       NaN
50412        NaN
141525     800.0
238016       NaN
190785       0.0
           ...  
354887       0.0
229420    3000.0
9192         NaN
180047       0.0
218327     788.0
139623     600.0
287084    2000.0
75960        0.0
301151    2900.0
340701    2500.0
124883       NaN
196079    1190.0
163698    1200.0
169020    2000.0
334240    1100.0
225419       0.0
311509    1300.0
232458     900.0
306966       NaN
48145        0.0
236146    1200.0
308073    1200.0
181200    1000.0
49756     1200.0
244        522.0
66286        0.0
323178     250.0
11417        N

### ANÁLISE DESCRITIVA

___
## MODELOS DE PREDIÇÃO

[O tópico C.PROCESSO E ESTATÍSTICAS DE VALIDAÇÃO estará dentro deste tópico. Organize-o aqui de forma clara!]

[ORGANIZE AQUI TODOS OS MODELOS DE PREDIÇÃO NECESSÁRIOS NESTE PROJETO!]

o MODELO DE PREDIÇÃO PELA MÉDIA

o MODELO DOS K VIZINHOS MAIS PRÓXIMOS (K-Nearest Neighbors Regression)

o MODELO DE REGRESSÃO LINEAR (Multiple Linear Regression)

o MODELO DE ÁRVORES DE REGRESSÃO (Decision Tree Regression)

___
## CONCLUSÃO

___
## REFERÊNCIAS BIBLIOGRÁFICAS